In [ ]:
from neuralforecast.losses.numpy import mape, mae
import pandas as pd
import numpy as np
from statistics import mean, stdev

In [ ]:

def sort_names(eval_df, experiment_name):
    if experiment_name == 'size_ablation':
        names = np.concatenate([
            eval_df.filter(regex='tiny').columns.values,
            eval_df.filter(regex='mini').columns.values,
            eval_df.filter(regex='small').columns.values,
            eval_df.filter(regex='base').columns.values,
                           ])

    elif experiment_name == 'tokenlen_ablation':
        names = np.concatenate([eval_df.filter(regex='len8').columns.values,
            eval_df.filter(regex='len16').columns.values,
            eval_df.filter(regex='len32').columns.values,
            eval_df.filter(regex='len64').columns.values,
            eval_df.filter(regex='len96').columns.values,
            eval_df.filter(regex='len128').columns.values,
                           ])

    elif experiment_name == 'pe_ablation':
        names = np.concatenate([
            eval_df.filter(regex='_relative').columns.values,
            eval_df.filter(regex='_sincos').columns.values,
            eval_df.filter(regex='rope').columns.values,
                           ])

    elif experiment_name == 'proj_ablation':
        names = np.concatenate([
            eval_df.filter(regex='linear').columns.values,
            eval_df.filter(regex='residual').columns.values,
                           ])

    elif experiment_name == 'scaler_ablation':
        names = np.concatenate([
            eval_df.filter(regex='standard').columns.values,
            eval_df.filter(regex='revin').columns.values,
            eval_df.filter(regex='robust').columns.values,
                           ])

    elif experiment_name == 'loss_ablation':
        names = np.concatenate([
            eval_df.filter(regex='mae').columns.values,
            eval_df.filter(regex='mse').columns.values,
            eval_df.filter(regex='huber').columns.values,
            eval_df.filter(regex='studentt').columns.values,
                           ])
        names = names[:4]

    elif experiment_name == 'attn_ablation':
        names = np.concatenate([
            eval_df.filter(regex='encoderb').columns.values,
            eval_df.filter(regex='encoderc').columns.values,
            eval_df.filter(regex='decoder').columns.values,
            eval_df.filter(regex='encoderdecoder').columns.values,
                                ])

    elif experiment_name == 'tokenization_ablation':
        names = np.concatenate([
            eval_df.filter(regex='none').columns.values,
            eval_df.filter(regex='patch').columns.values,
            eval_df.filter(regex='lag').columns.values,
            eval_df.filter(regex='bin').columns.values,
                                ])
        names = names[:3]

    elif experiment_name == 'contextlen_ablation':
        names = np.concatenate([
            eval_df.filter(regex='256').columns.values,
            eval_df.filter(regex='512').columns.values,
                                ])
        names = names[:3]

    elif experiment_name == 'decomp_ablation':
        names = np.concatenate([
            eval_df.filter(regex='tsdecomp').columns.values,
            eval_df.filter(regex='fourierdecomp').columns.values,
                                ])
        names = names[:3]

    elif experiment_name == 'nont5models':
        names = np.concatenate([
            eval_df.filter(regex='dlinear').columns.values,
            eval_df.filter(regex='nlinear').columns.values,
            eval_df.filter(regex='mlp').columns.values,
            eval_df.filter(regex='nhits').columns.values,
            eval_df.filter(regex='nbeats').columns.values,
            eval_df.filter(regex='tsmixer').columns.values,
            eval_df.filter(regex='lstm').columns.values,
            eval_df.filter(regex='tcn').columns.values,
            eval_df.filter(regex='timesnet').columns.values,
            eval_df.filter(regex='tcn').columns.values,
            eval_df.filter(regex='vanillatransformer').columns.values,
            eval_df.filter(regex='itransformer').columns.values,
            eval_df.filter(regex='autoformer').columns.values,
            eval_df.filter(regex='informer').columns.values,
            eval_df.filter(regex='tft').columns.values,
            eval_df.filter(regex='patchtst').columns.values,
                           ])

    return names

In [ ]:
def get_results(dataset_name, n_compositions, experiment_name, experiment_mode):
    
    save_dir = 'ADD HERE'
    n_series = 100
    ood_ratio = 0.0
    
    save_path1 = f'{save_dir}/results_randomseed1/{experiment_name}/{dataset_name}_{n_compositions}bases/{experiment_mode}/{n_series}_samples/{experiment_mode}_model_aggregate_dataset_fcsts.csv'
    eval_df1 = pd.read_csv(save_path1)
    eval_df1.set_index(['unique_id', 'ds'], inplace=True)
    
    save_path2 = f'{save_dir}/results_randomseed5/{experiment_name}/{dataset_name}_{n_compositions}bases/{experiment_mode}/{n_series}_samples/{experiment_mode}_model_aggregate_dataset_fcsts.csv'
    eval_df2 = pd.read_csv(save_path2)
    eval_df2.set_index(['unique_id', 'ds'], inplace=True)
    
    save_path3 = f'{save_dir}/results_randomseed10/{experiment_name}/{dataset_name}_{n_compositions}bases/{experiment_mode}/{n_series}_samples/{experiment_mode}_model_aggregate_dataset_fcsts.csv'
    eval_df3 = pd.read_csv(save_path3)
    eval_df3.set_index(['unique_id', 'ds'], inplace=True)
    
    dict1 = {}
    dict2 = {}
    dict3 = {}
    names = sort_names(eval_df1, experiment_name)
    for col in names:
        dict1[col] = mae(eval_df1[col], eval_df1['y'])
        dict2[col] = mae(eval_df2[col], eval_df2['y'])
        dict3[col] = mae(eval_df3[col], eval_df3['y'])
    dicts = [dict1, dict2, dict3]

    mean_result = {}
    stdev_result = {}
    for key in names:
        values = [d[key] for d in dicts]
        mean_result[key] = round(mean(values), 3)
        stdev_result[key] = round(stdev(values), 3)

    return mean_result, stdev_result


## Non-T5 Model Results

In [ ]:
dataset_names = ['synthetic_sinusoid_composition',
           'ecl_100_series',
           'ettm2_100_series',
           'solar1h_100_series',
           'subseasonal_100_series',
           'loopseattle_100_series'
           ]

n_comps = [2, 100, 100, 100, 100, 100]

baseline_results = pd.DataFrame()
for dataset_name, n_compositions in zip(dataset_names, n_comps):
    mean_result, stdev_result = get_results(
        dataset_name,
        n_compositions,
        'baselines',
        experiment_mode='component')
    col1 = pd.DataFrame([mean_result]).T
    #col1 = pd.DataFrame([stdev_result]).T
    
    mean_result, stdev_result = get_results(
        dataset_name,
        n_compositions,
        'baselines',
        experiment_mode='aggregate')
    col2 = pd.DataFrame([mean_result]).T
    #col2 = pd.DataFrame([stdev_result]).T

    baseline_results = pd.concat([baseline_results, col1, col2], axis=1)

In [ ]:
baseline_results

## Ablation Results

In [ ]:
dataset_name = 'loopseattle_100_series'
n_compositions = 100
experiment_mode = 'aggregate' #'component' #

mean_result, stdev_result = get_results(
    dataset_name,
    n_compositions,
    'tokenization_ablation', # Replace with ablation experiment name
    experiment_mode=experiment_mode)

for (mkey, mvalue), (skey, svalue) in zip(mean_result.items(), stdev_result.items()):
    print(f"{mkey, skey}: {mvalue} ({svalue})")
